In [58]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk import ngrams
import json


In [59]:
stop_words = set(stopwords.words('english'))
# this function returns a list of tokenized and stemmed words of any text
def get_tokenized_list(doc_text):
    tokens = nltk.word_tokenize(doc_text)
    return tokens

# This function will performing stemming on tokenized words
def word_stemmer(token_list):
  ps = nltk.stem.PorterStemmer()
  stemmed = []
  for words in token_list:
    stemmed.append(ps.stem(words))
  return stemmed

# Function to remove stopwords from tokenized word list
def remove_stopwords(doc_text):
  cleaned_text = []
  for words in doc_text:
    if words not in stop_words:
      cleaned_text.append(words)
  return cleaned_text

In [60]:
corpus = [
        'Home team will win the match because away team has a lot of injured players',
        'Away team is in good health!',
        'Both teams are in a bad situation. They both lost last three matches',
        'Home team is in good health!',
        'Home team won last four matches while away team lost last two matches',
        'We expected home team and away team will make at least one goal'
    ]

In [61]:
corpu = pd.read_csv('completati/description_predictions.csv', index_col=0)

with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

corpus = []
for cor, c in corpu.iterrows():
    h_team, a_team, description = c.h_team[0], c.a_team[0], c.description[0]
    
    syn = {}
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 
    
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    corpus.append(description)

    

In [62]:
cleaned_corpus = []
for doc in corpus:
  tokens = get_tokenized_list(doc)
  doc_text = remove_stopwords(tokens)
  doc_text  = word_stemmer(doc_text)
  #doc_text  = word_stemmer(tokens)
  doc_text = ' '.join(doc_text)
  cleaned_corpus.append(doc_text)


In [ ]:
vectorizerX = TfidfVectorizer()
vectorizerX.fit(cleaned_corpus) #impara il vocabolario e idf dal training set
doc_vector = vectorizerX.transform(cleaned_corpus) #trasforma i documenti in una matrice document-term

In [ ]:
df1 = pd.DataFrame(doc_vector.toarray(), columns=vectorizerX.get_feature_names())

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(doc_vector[:200])
#print(kmeans.labels_)
#print(kmeans.predict(doc_vector[200:]))
lett = []
for km in kmeans.labels_:
    if km == 0:
        lett.append('P')
    if km == 1:
        lett.append('V')
    if km == 2:
        lett.append('N')